In [4]:
import pandas as pd
import numpy as np

In [5]:
print('Hello Capstone Project Course')

Hello Capstone Project Course


# <font color = purple>__1. Segmenting and Clustering Neighborhoods in Toronto__</font>

In [6]:

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [7]:
from pandas.io.html import read_html

link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" 
wikitable = read_html(link,attrs={'class':'wikitable sortable'}) # I'm going for the easiest way using the 'read_html' pandas method to scrap the wikitable

print('Extracted {num} wikitables'.format(num=len(wikitables))) # Check whether there are more than one table on the link


Extracted 1 wikitables


In [8]:
table = wikitables[0]
table.head() 

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
table = table[table.Borough != 'Not assigned'] # Remove rows where there is no value assigned to the column 'Borough'
table.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [10]:
table.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'},inplace=True) # Rename column as required on the casptone 
table.head(15)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [11]:
table['Neighborhood'] = table['Neighborhood'].replace('Not assigned',table['Borough']) # Replace NAN value assigned in 'Neighborhood' with the value assigned on' Borough'

table.head(15)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [122]:
table.shape # Check the shape prior to the groupBy function

(211, 3)

In [25]:
dfGroupby = table.groupby(['PostalCode','Borough'])['Neighborhood'].apply(list) # I'm creating a view of my first df using the group by function in order to group neighborhood under a unique PC
dfGroupby = dfGroupby.sample(frac=1).reset_index() # reset the index to avoid problems
dfGroupby['Neighborhood'] = dfGroupby['Neighborhood'].str.join(', ') # Add a coma to separates the n values in the column 'Nighborhood' 

dfGroupby.head(15)
    

,PostalCode,Borough,Neighborhood
0,M4E,East Toronto,The Beaches
1,M7A,Queen's Park,Queen's Park
2,M3B,North York,Don Mills North
3,M4Y,Downtown Toronto,Church and Wellesley
4,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ..."
5,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
6,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."
7,M2K,North York,Bayview Village
8,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
9,M6A,North York,"Lawrence Heights, Lawrence Manor"


In [26]:
dfGroupby.shape # New shape of my df

(103, 3)

In [27]:
url = 'http://cocl.us/Geospatial_data'
url_data = pd.read_csv(url) # loading the df from the file on my Notebook using the read_csv method
url_data.rename(columns={'Postal Code': 'PostalCode'},inplace=True) # updating the label to have similar label 

url_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
Lastdf = dfGroupby.merge(url_data) # using the merge method to 
Lastdf.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
2,M3B,North York,Don Mills North,43.745906,-79.352188
3,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
4,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
5,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
6,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999
7,M2K,North York,Bayview Village,43.786947,-79.385975
8,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
9,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763


,PostalCode,Borough,Neighborhood,Latitude,Longitude
34,M2H,North York,Hillcrest Village,43.803762,-79.363452
